<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="900px"/> 

<a id="title_ID"></a>
# MIRI MRS Batch Processing Notebook #

**Author**: David Law, AURA Associate Astronomer, MIRI branch
<br>
**Last Updated**: October 14, 2022
<br>
**Pipeline Version**: 1.8.0+

The purpose of this notebook is to provide a framework for batch processing of MIRI MRS data through all three pipeline stages.  Data is assumed to be located in two Observation folders (science and background) according to paths set up below.

This example is set up to use observations from APT program 2732 (Stephan's Quintet ERO observations) using a standard 4-pt dither with dedicated background in all three grating settings.  Input data for this notebook can be obtained by downloading the 'uncal' files from MAST and placing them in directories set up below.  

Changes:<br>
Sep 1 2022: Add some commentary and example on how to use multicore processing in Detector 1<br>
Sep 12 2022: Disable unnecessary cube/1d spectra production for individual science exposures in Spec 2<br>
Oct 14 2022: Include residual fringe correction in spec2 (note that this will CRASH earlier pipeline versions!

<hr style="border:1px solid gray"> </hr>

1.<font color='white'>-</font>Configuration <a class="anchor" id="intro"></a>
------------------

In [1]:
# Set parameters to be changed here.
# It should not be necessary to edit cells below this in general unless modifying pipeline processing steps.

import sys,os, pdb

# CRDS context (if overriding)
#%env CRDS_CONTEXT jwst_0771.pmap

# Point to where the uncalibrated FITS files are from the science observation
input_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/'

# Point to where you want the output science results to go
output_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/'

# Point to where the uncalibrated FITS files are from the background observation
# If no background observation, leave this blank
input_bgdir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/'

# Point to where the output background observations should go
# If no background observation, leave this blank
output_bgdir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/'

# Whether or not to run a given pipeline stage
# Science and background are processed independently through det1+spec2, and jointly in spec3

# Science processing
dodet1=True
dospec2=True
dospec3=True

# Background processing
dodet1bg=True
dospec2bg=True

# If there is no background folder, ensure we don't try to process it
if (input_bgdir == ''):
    dodet1bg=False
    dospec2bg=False

<hr style="border:1px solid gray"> </hr>

2.<font color='white'>-</font>Imports and setup <a class="anchor" id="intro"></a>
------------------

In [2]:
# Now let's use the entire available screen width for the notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# Basic system utilities for interacting with files
import glob
import time
import shutil
import warnings
import zipfile
import urllib.request

# Astropy utilities for opening FITS and ASCII files
from astropy.io import fits
from astropy.io import ascii
from astropy.utils.data import download_file
# Astropy utilities for making plots
from astropy.visualization import (LinearStretch, LogStretch, ImageNormalize, ZScaleInterval)

# Numpy for doing calculations
import numpy as np

# Matplotlib for making plots
import matplotlib.pyplot as plt
from matplotlib import rc

In [4]:
# Import the base JWST package
import jwst

In [5]:
# JWST pipelines (encompassing many steps)
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# JWST pipeline utilities
from jwst import datamodels # JWST datamodels
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

from stcal import dqflags # Utilities for working with the data quality (DQ) arrays

In [6]:
# Output subdirectories to keep science data products organized
# Note that the pipeline might complain about this as it is intended to work with everything in a single
# directory, but it nonetheless works fine for the examples given here.
det1_dir = os.path.join(output_dir, 'stage1/') # Detector1 pipeline outputs will go here
spec2_dir = os.path.join(output_dir, 'stage2/') # Spec2 pipeline outputs will go here
spec3_dir = os.path.join(output_dir, 'stage3/') # Spec3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(spec2_dir):
    os.makedirs(spec2_dir)
if not os.path.exists(spec3_dir):
    os.makedirs(spec3_dir)

In [7]:
# Output subdirectories to keep background data products organized
det1_bgdir = os.path.join(output_bgdir, 'stage1/') # Detector1 pipeline outputs will go here
spec2_bgdir = os.path.join(output_bgdir, 'stage2/') # Spec2 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if (output_bgdir != ''):
    if not os.path.exists(det1_bgdir):
        os.makedirs(det1_bgdir)
    if not os.path.exists(spec2_bgdir):
        os.makedirs(spec2_bgdir)

In [8]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

<hr style="border:1px solid gray"> </hr>

3.<font color='white'>-</font>Detector1 Pipeline <a class="anchor" id="det1"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Detector1 pipeline to create Lvl2a data products (i.e., uncalibrated slope images).
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html
</div>

In [9]:
# First we'll define a function that will call the detector1 pipeline with our desired set of parameters
# We won't enumerate the individual steps
def rundet1(filename, outdir):
    print(filename)
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = outdir # Specify where the output should go
    
    # Overrides for whether or not certain steps should be skipped
    #det1.dq_init.skip = False
    #det1.saturation.skip = False
    #det1.firstframe.skip = False
    #det1.lastframe.skip = False
    #det1.reset.skip = False
    #det1.linearity.skip = False
    #det1.rscd.skip = False
    #det1.dark_current.skip = False
    #det1.refpix.skip = False
    #det1.jump.skip = False
    #det1.ramp_fit.skip = False
    #det1.gain_scale.skip = False
    
    # The jump and ramp fitting steps can benefit from multi-core processing, but this is off by default
    # Turn them on here if desired by choosing how many cores to use (quarter, half, or all)
    det1.jump.maximum_cores='half'
    det1.ramp_fit.maximum_cores='half'
    
    # Bad pixel mask overrides
    #det1.dq_init.override_mask = 'myfile.fits'

    # Saturation overrides
    #et1.saturation.override_saturation = 'myfile.fits'
    
    # Reset overrides
    #det1.reset.override_reset = 'myfile.fits'
        
    # Linearity overrides
    #det1.linearity.override_linearity = 'myfile.fits'

    # RSCD overrides
    #det1.rscd.override_rscd = 'myfile.fits'
        
    # DARK overrides
    #det1.dark_current.override_dark = 'myfile.fits'
        
    # GAIN overrides
    #det1.jump.override_gain = 'myfile.fits'
    #det1.ramp_fit.override_gain = 'myfile.fits'
                
    # READNOISE overrides
    #det1.jump.override_readnoise = 'myfile.fits'
    #det1.ramp_fit.override_readnoise = 'myfile.fits'
        
    det1.save_results = True # Save the final resulting _rate.fits files
    det1(filename) # Run the pipeline on an input list of files

In [10]:
# Now let's look for input files of the form *uncal.fits from the science observation
sstring = input_dir + 'jw*mirifu*uncal.fits'
lvl1b_files = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1b_files)) + ' science input files to process')

Found 24 science input files to process


In [11]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1:
    for file in lvl1b_files:
        rundet1(file, det1_dir)
else:
    print('Skipping Detector1 processing')

2022-10-14 13:15:58,730 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-10-14 13:15:58,732 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-10-14 13:15:58,734 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-10-14 13:15:58,736 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-10-14 13:15:58,738 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-10-14 13:15:58,740 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-10-14 13:15:58,742 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-10-14 13:15:58,744 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-10-14 13:15:58,746 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-10-14 13:15:58,748 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-10-14 1

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/jw02732004001_02101_00001_mirifulong_uncal.fits


2022-10-14 13:16:03,666 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02101_00001_mirifulong_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-10-14 13:16:04,383 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0083.fits'.
2022-10-14 13:16:04,385 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits'.
2022-10-14 13:16:04,386 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is 'N/A'.
2022-10-14 13:16:04,387 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0033.fits'.
2022-10-14 13:16:04,388 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/grp/crds/jwst/

2022-10-14 13:16:20,387 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02101_00001_mirifulong_uncal.fits>,).
2022-10-14 13:16:20,389 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal'}
2022-10-14 13:16:20,428 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0033.fits
2022-10-14 13:16:22,277 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-10-14 13:16:22,427 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02101_

2022-10-14 13:20:43,953 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-10-14 13:20:43,953 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-10-14 13:20:44,049 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-10-14 13:20:44,049 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-10-14 13:20:44,638 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-10-14 13:20:44,639 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-10-14 13:20:44,641 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of processors used for multiprocessing: 6
2022-10-14 13:21:37,582 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-10-14 13:21:37,765 - s

...

2022-10-14 15:23:58,442 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-10-14 15:23:58,442 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-10-14 15:23:58,529 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-10-14 15:23:58,530 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-10-14 15:23:59,057 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-10-14 15:23:59,058 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-10-14 15:23:59,059 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of processors used for multiprocessing: 6
2022-10-14 15:24:43,926 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-10-14 15:24:44,115 - s

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/jw02732004001_02105_00004_mirifushort_uncal.fits


2022-10-14 15:24:47,999 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02105_00004_mirifushort_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-10-14 15:24:48,008 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0081.fits'.
2022-10-14 15:24:48,010 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits'.
2022-10-14 15:24:48,011 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is 'N/A'.
2022-10-14 15:24:48,012 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits'.
2022-10-14 15:24:48,013 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/grp/crds/jwst

2022-10-14 15:25:03,124 - stpipe.Detector1Pipeline.reset - INFO - Step reset done
2022-10-14 15:25:03,320 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02105_00004_mirifushort_uncal.fits>,).
2022-10-14 15:25:03,322 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal'}
2022-10-14 15:25:03,361 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits
2022-10-14 15:25:04,647 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-10-14 15:25:04,825 - stpipe.Detector1Pipeline.rscd - INFO - S

2022-10-14 15:29:17,005 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0087.fits
2022-10-14 15:29:17,006 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits
2022-10-14 15:29:17,101 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-10-14 15:29:17,101 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-10-14 15:29:17,681 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-10-14 15:29:17,683 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-10-14 15:29:17,684 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of processors used for multiprocessing: 6
2022-10-14 15:30:15,695 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-10-14 15:30:15,898 - s

In [12]:
# Now let's look for input files of the form *uncal.fits from the background observation
sstring = input_bgdir + 'jw*mirifu*uncal.fits'
lvl1b_files = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1b_files)) + ' background input files to process')

Found 12 background input files to process


In [13]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1bg:
    for file in lvl1b_files:
        rundet1(file, det1_bgdir)
else:
    print('Skipping Detector1 processing')

2022-10-14 15:30:17,299 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-10-14 15:30:17,301 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-10-14 15:30:17,303 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-10-14 15:30:17,304 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-10-14 15:30:17,306 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-10-14 15:30:17,308 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-10-14 15:30:17,310 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-10-14 15:30:17,311 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-10-14 15:30:17,313 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-10-14 15:30:17,315 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-10-14 1

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/jw02732005001_02101_00001_mirifulong_uncal.fits


2022-10-14 15:30:20,000 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02101_00001_mirifulong_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-10-14 15:30:20,016 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0083.fits'.
2022-10-14 15:30:20,017 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits'.
2022-10-14 15:30:20,019 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is 'N/A'.
2022-10-14 15:30:20,020 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0033.fits'.
2022-10-14 15:30:20,022 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/grp/crds/jwst/

2022-10-14 15:30:35,472 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 40, 1024, 1032) from jw02732005001_02101_00001_mirifulong_uncal.fits>,).
2022-10-14 15:30:35,474 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal'}
2022-10-14 15:30:35,511 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0033.fits
2022-10-14 15:30:37,167 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-10-14 15:30:37,354 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(2, 40, 1024, 1032) from jw02732005001_02101_

2022-10-14 15:34:54,282 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-10-14 15:34:54,283 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-10-14 15:34:54,373 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-10-14 15:34:54,374 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-10-14 15:34:54,943 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-10-14 15:34:54,947 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-10-14 15:34:54,948 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of processors used for multiprocessing: 6
2022-10-14 15:35:46,329 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-10-14 15:35:46,526 - s

...

2022-10-14 16:29:41,676 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-10-14 16:29:41,676 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-10-14 16:29:41,767 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-10-14 16:29:41,768 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-10-14 16:29:42,313 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-10-14 16:29:42,315 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-10-14 16:29:42,318 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of processors used for multiprocessing: 6
2022-10-14 16:30:36,533 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-10-14 16:30:36,729 - s

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/jw02732005001_02105_00002_mirifushort_uncal.fits


2022-10-14 16:30:40,613 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02105_00002_mirifushort_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-10-14 16:30:40,626 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0081.fits'.
2022-10-14 16:30:40,627 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits'.
2022-10-14 16:30:40,628 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is 'N/A'.
2022-10-14 16:30:40,629 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits'.
2022-10-14 16:30:40,631 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/grp/crds/jwst

2022-10-14 16:30:55,787 - stpipe.Detector1Pipeline.reset - INFO - Step reset done
2022-10-14 16:30:55,978 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 40, 1024, 1032) from jw02732005001_02105_00002_mirifushort_uncal.fits>,).
2022-10-14 16:30:55,980 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal'}
2022-10-14 16:30:56,019 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits
2022-10-14 16:30:57,356 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-10-14 16:30:57,546 - stpipe.Detector1Pipeline.rscd - INFO - S

2022-10-14 16:35:24,605 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0087.fits
2022-10-14 16:35:24,606 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits
2022-10-14 16:35:24,694 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-10-14 16:35:24,695 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-10-14 16:35:25,214 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-10-14 16:35:25,215 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-10-14 16:35:25,216 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of processors used for multiprocessing: 6
2022-10-14 16:36:16,635 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-10-14 16:36:16,829 - s

In [14]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 12019.5052 seconds


4.<font color='white'>-</font>Spec2 Pipeline <a class="anchor" id="spec2"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Spec2 pipeline in order to produce Lvl2b data products (i.e., calibrated slope images and quick-look data cubes and 1d spectra).  
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html
</div>

In [15]:
# Define a function that will call the spec2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def runspec2(filename, outdir, nocubes=False):
    spec2 = Spec2Pipeline()
    spec2.output_dir = outdir

    # Assign_wcs overrides
    #spec2.assign_wcs.override_distortion = 'myfile.asdf'
    #spec2.assign_wcs.override_regions = 'myfile.asdf'
    #spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    #spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Flatfield overrides
    #spec2.flat.override_flat = 'myfile.fits'
        
    # Straylight overrides
    #spec2.straylight.override_mrsxartcorr = 'myfile.fits'
        
    # Fringe overrides
    #spec2.fringe.override_fringe = 'myfile.fits'
    
    # Photom overrides
    #spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    #spec2.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec2.extract1d.override_extract1d = 'myfile.asdf'
    #spec2.extract1d.override_apcorr = 'myfile.asdf'
        
    # Overrides for whether or not certain steps should be skipped
    #spec2.assign_wcs.skip = False
    #spec2.bkg_subtract.skip = True
    #spec2.flat_field.skip = False
    #spec2.srctype.skip = False
    #spec2.straylight.skip = False
    #spec2.fringe.skip = False
    #spec2.photom.skip = False
    spec2.residual_fringe.skip = False
    #spec2.cube_build.skip = False
    #spec2.extract_1d.skip = False
    
    # This nocubes option allows us to skip the cube building and 1d spectral extraction for individual
    # science data frames, but run it for the background data (as the 1d spectra are needed later
    # for the master background step in Spec3)
    if (nocubes):
        spec2.cube_build.skip = True
        spec2.extract_1d.skip = True
    
    # Some cube building options
    #spec2.cube_build.weighting='drizzle'
    #spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky
      
    spec2.save_results = True
    spec2(filename)

In [16]:
# Look for uncalibrated science slope files from the Detector1 pipeline
sstring = det1_dir + 'jw*mirifu*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 24 input files to process


In [17]:
if dospec2:
    for file in ratefiles:
        runspec2(file, spec2_dir, nocubes=True)
else:
    print('Skipping Spec2 processing')

2022-10-14 16:36:18,304 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2022-10-14 16:36:18,308 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-10-14 16:36:18,310 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-10-14 16:36:18,312 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2022-10-14 16:36:18,314 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2022-10-14 16:36:18,316 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2022-10-14 16:36:18,322 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2022-10-14 16:36:18,324 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2022-10-14 16:36:18,326 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2022-10-14 16:36:18,328 - stpipe.Spec2Pipeline.

2022-10-14 16:36:18,785 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02101_00001_mirifulong_rate.fits' reftypes = ['area', 'barshadow', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'fringefreq', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'specwcs', 'wavecorr', 'wavelengthrange', 'wfssbkg']
2022-10-14 16:36:18,810 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-10-14 16:36:18,810 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-10-14 16:36:18,811 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2022-10-14 16:36:18,812 - stpipe.Spec2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2022-10-14 16:36:18,812 - stpipe.Spec2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A'

2022-10-14 16:36:25,918 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1'}
2022-10-14 16:36:25,919 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step skipped.
2022-10-14 16:36:25,924 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging done
2022-10-14 16:36:26,061 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype running with args (<IFUImageModel(1024, 1032) from jw02732004001_02101_00001_mirifulong_rate.fits>,).
2022-10-14 16:36:26,063 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_inde

2022-10-14 16:36:55,351 - stpipe.Spec2Pipeline.residual_fringe - INFO - Using FRINGEFREQ reference file:/grp/crds/jwst/references/jwst/jwst_miri_fringefreq_0003.fits
2022-10-14 16:36:55,371 - stpipe.Spec2Pipeline.residual_fringe - INFO - Using MRS regions reference file: /grp/crds/jwst/references/jwst/jwst_miri_regions_0107.asdf
2022-10-14 16:36:58,414 - stpipe.Spec2Pipeline.residual_fringe - INFO -  Using 8 throughput threshold.
2022-10-14 16:36:58,415 - stpipe.Spec2Pipeline.residual_fringe - INFO - Detector mirifulong [3, 4] 
2022-10-14 16:36:58,500 - stpipe.Spec2Pipeline.residual_fringe - INFO - Processing channel 3
2022-10-14 16:36:58,531 - stpipe.Spec2Pipeline.residual_fringe - INFO - Number of slices in band 16 
2022-10-14 16:36:59,294 - stpipe.Spec2Pipeline.residual_fringe - INFO -  Processing slice 301.0 =================================
2022-10-14 16:36:59,309 - stpipe.Spec2Pipeline.residual_fringe - WARNING - /Users/dlaw/jwcode/jwst/jwst/residual_fringe/residual_fringe.py:213

2022-10-14 16:40:26,208 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2
2022-10-14 16:40:26,208 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_1001.pmap
2022-10-14 16:40:27,809 - stpipe.Spec2Pipeline - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage2/jw02732004001_02101_00001_mirifulong_cal.fits
2022-10-14 16:40:27,810 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done
2022-10-14 16:40:27,832 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2022-10-14 16:40:27,834 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-10-14 16:40:27,837 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-10-14 16:40:27,838 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2022-10-14 16:40:27,840 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2022-10-14 16:40:27,842 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2d

...

2022-10-14 18:33:31,006 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': True, 'suffix': 'x1d', 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1', 'smoothing_length': None, 'bkg_fit': None, 'bkg_order': None, 'bkg_sigma_clip': 3.0, 'log_increment': 50, 'subtract_background': None, 'use_source_posn': None, 'center_xy': None, 'apply_apcorr': True, 'soss_threshold': 0.01, 'soss_n_os': 2, 'soss_transform': None, 'soss_tikfac': None, 'soss_width': 40.0, 'soss_bad_pix': 'model', 'soss_modelname': None}
2022-10-14 18:33:31,007 - stpipe.Spec2Pipeline.extract_1d - INFO - Step skipped.
2022-10-14 18:33:31,012 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done
2022-10-14 18:33:31,012 - stpipe.Spec2Pipeline - INFO - Finished proce

2022-10-14 18:33:33,288 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02105_00004_mirifushort_rate.fits' reftypes = ['area', 'barshadow', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'fringefreq', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'specwcs', 'wavecorr', 'wavelengthrange', 'wfssbkg']
2022-10-14 18:33:33,315 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-10-14 18:33:33,316 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-10-14 18:33:33,316 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2022-10-14 18:33:33,317 - stpipe.Spec2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2022-10-14 18:33:33,318 - stpipe.Spec2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A

2022-10-14 18:33:41,416 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1'}
2022-10-14 18:33:41,417 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step skipped.
2022-10-14 18:33:41,421 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging done
2022-10-14 18:33:41,588 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype running with args (<IFUImageModel(1024, 1032) from jw02732004001_02105_00004_mirifushort_rate.fits>,).
2022-10-14 18:33:41,589 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_ind

2022-10-14 18:34:17,893 - stpipe.Spec2Pipeline.residual_fringe - INFO - Step residual_fringe parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'residual_fringe', 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1', 'save_intermediate_results': False, 'ignore_region_min': None, 'ignore_region_max': None}
2022-10-14 18:34:17,925 - stpipe.Spec2Pipeline.residual_fringe - INFO - Using FRINGEFREQ reference file:/grp/crds/jwst/references/jwst/jwst_miri_fringefreq_0004.fits
2022-10-14 18:34:17,943 - stpipe.Spec2Pipeline.residual_fringe - INFO - Using MRS regions reference file: /grp/crds/jwst/references/jwst/jwst_miri_regions_0101.asdf
2022-10-14 18:34:21,886 - stpipe.Spec2Pipeline.residual_fringe - INFO -  Using 8 throughput threshold.
2022-10-14 18:34:21,887 - stpipe.Spec2Pipelin

2022-10-14 18:36:14,229 - stpipe.Spec2Pipeline.cube_build - INFO - Step skipped.
2022-10-14 18:36:14,234 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build done
2022-10-14 18:36:14,445 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage2/jw02732004001_02105_00004_mirifushort_cal.fits>,).
2022-10-14 18:36:14,448 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': True, 'suffix': 'x1d', 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1', 'smoothing_length': None, 'bkg_fit': None, 'bkg_order': None, 'bkg_sigma_clip': 3.0, 'log_increment': 50, 'subtract_background': None, 'use_source_posn': None, 'center_xy': None, 'app

In [18]:
# Look for uncalibrated background slope files from the Detector1 pipeline
sstring = det1_bgdir + 'jw*mirifu*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 12 input files to process


In [19]:
if dospec2bg:
    for file in ratefiles:
        runspec2(file, spec2_bgdir)
else:
    print('Skipping Spec2 processing')

2022-10-14 18:36:16,519 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2022-10-14 18:36:16,521 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-10-14 18:36:16,523 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-10-14 18:36:16,525 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2022-10-14 18:36:16,526 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2022-10-14 18:36:16,528 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2022-10-14 18:36:16,533 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2022-10-14 18:36:16,535 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2022-10-14 18:36:16,537 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2022-10-14 18:36:16,538 - stpipe.Spec2Pipeline.

2022-10-14 18:36:17,197 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02101_00001_mirifulong_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'fringefreq', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']
2022-10-14 18:36:17,234 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-10-14 18:36:17,236 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-10-14 18:36:17,236 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-10-14 18:36:17,237 - stpipe.Spec2Pipeline - INFO - Prefet

2022-10-14 18:36:23,716 - stpipe.Spec2Pipeline.bkg_subtract - INFO - Step bkg_subtract done
2022-10-14 18:36:23,872 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract running with args (<IFUImageModel(1024, 1032) from jw02732005001_02101_00001_mirifulong_rate.fits>, []).
2022-10-14 18:36:23,873 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage1'}
2022-10-14 18:36:23,874 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-10-14 18:36:23,879 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-10-14 18:36:24,036 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with arg

2022-10-14 18:36:50,864 - stpipe.Spec2Pipeline.photom - INFO -  detector: MIRIFULONG
2022-10-14 18:36:50,865 - stpipe.Spec2Pipeline.photom - INFO -  exp_type: MIR_MRS
2022-10-14 18:36:50,865 - stpipe.Spec2Pipeline.photom - INFO -  band: SHORT
2022-10-14 18:36:51,807 - stpipe.Spec2Pipeline.photom - WARNING - Keyword CDP_LOW_QUAL does not correspond to an existing DQ mnemonic, so will be ignored
2022-10-14 18:36:51,843 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-10-14 18:36:52,080 - stpipe.Spec2Pipeline.residual_fringe - INFO - Step residual_fringe running with args (<IFUImageModel(1024, 1032) from jw02732005001_02101_00001_mirifulong_rate.fits>,).
2022-10-14 18:36:52,081 - stpipe.Spec2Pipeline.residual_fringe - INFO - Step residual_fringe parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'residual_fringe',

2022-10-14 18:37:08,858 - stpipe.Spec2Pipeline.cube_build - INFO - Reading cube parameter file /grp/crds/jwst/references/jwst/jwst_miri_cubepar_0009.fits
2022-10-14 18:37:09,490 - stpipe.Spec2Pipeline.cube_build - INFO - Output IFUcube are constructed from all the data 
2022-10-14 18:37:09,491 - stpipe.Spec2Pipeline.cube_build - INFO - Number of IFU cubes produced by this run = 1
2022-10-14 18:37:09,502 - stpipe.Spec2Pipeline.cube_build - INFO - Increasing spatial region of interest default value set for 4 dithers nan
2022-10-14 18:37:09,506 - stpipe.Spec2Pipeline.cube_build - INFO - Cube Geometry:
2022-10-14 18:37:09,507 - stpipe.Spec2Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(arcsec)  Min & Max (xi, eta arcsec)
2022-10-14 18:37:09,508 - stpipe.Spec2Pipeline.cube_build - INFO - Axis 1    51  26.00 -20.96792777   0.20000000  -5.10000008   5.10000008
2022-10-14 18:37:09,508 - stpipe.Spec2Pipeline.cube_build - INFO - Axis 2    53  27.00  33.99461152   0.20000000

...

2022-10-14 19:21:48,656 - stpipe.Spec2Pipeline.cube_build - INFO - The desired cubes cover the MIRI Channels: ['3', '4']
2022-10-14 19:21:48,656 - stpipe.Spec2Pipeline.cube_build - INFO - The desired cubes cover the MIRI subchannels: ['long', 'long']
2022-10-14 19:21:48,657 - stpipe.Spec2Pipeline.cube_build - INFO - Reading cube parameter file /grp/crds/jwst/references/jwst/jwst_miri_cubepar_0009.fits
2022-10-14 19:21:49,275 - stpipe.Spec2Pipeline.cube_build - INFO - Output IFUcube are constructed from all the data 
2022-10-14 19:21:49,276 - stpipe.Spec2Pipeline.cube_build - INFO - Number of IFU cubes produced by this run = 1
2022-10-14 19:21:49,286 - stpipe.Spec2Pipeline.cube_build - INFO - Increasing spatial region of interest default value set for 4 dithers nan
2022-10-14 19:21:49,290 - stpipe.Spec2Pipeline.cube_build - INFO - Cube Geometry:
2022-10-14 19:21:49,290 - stpipe.Spec2Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(arcsec)  Min & Max (xi, eta arcsec)


2022-10-14 19:22:44,021 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02105_00002_mirifushort_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'fringefreq', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']
2022-10-14 19:22:44,068 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-10-14 19:22:44,069 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-10-14 19:22:44,070 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-10-14 19:22:44,070 - stpipe.Spec2Pipeline - INFO - Prefe

2022-10-14 19:22:52,028 - stpipe.Spec2Pipeline.bkg_subtract - INFO - Step bkg_subtract done
2022-10-14 19:22:52,208 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract running with args (<IFUImageModel(1024, 1032) from jw02732005001_02105_00002_mirifushort_rate.fits>, []).
2022-10-14 19:22:52,209 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage1'}
2022-10-14 19:22:52,210 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-10-14 19:22:52,215 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-10-14 19:22:52,393 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with ar

2022-10-14 19:23:26,229 - stpipe.Spec2Pipeline.photom - INFO - Using photom reference file: /grp/crds/jwst/references/jwst/jwst_miri_photom_0080.fits
2022-10-14 19:23:26,230 - stpipe.Spec2Pipeline.photom - INFO - Using area reference file: N/A
2022-10-14 19:23:28,060 - stpipe.Spec2Pipeline.photom - INFO - Using instrument: MIRI
2022-10-14 19:23:28,060 - stpipe.Spec2Pipeline.photom - INFO -  detector: MIRIFUSHORT
2022-10-14 19:23:28,061 - stpipe.Spec2Pipeline.photom - INFO -  exp_type: MIR_MRS
2022-10-14 19:23:28,062 - stpipe.Spec2Pipeline.photom - INFO -  band: LONG
2022-10-14 19:23:28,963 - stpipe.Spec2Pipeline.photom - WARNING - Keyword CDP_LOW_QUAL does not correspond to an existing DQ mnemonic, so will be ignored
2022-10-14 19:23:28,998 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-10-14 19:23:29,294 - stpipe.Spec2Pipeline.residual_fringe - INFO - Step residual_fringe running with args (<IFUImageModel(1024, 1032) from jw02732005001_02105_00002_mirifushort_rate.fits>,

2022-10-14 19:23:37,166 - stpipe.Spec2Pipeline.residual_fringe - INFO -  Processing slice 216.0 =================================
2022-10-14 19:23:37,218 - stpipe.Spec2Pipeline.residual_fringe - INFO -  Processing slice 217.0 =================================
2022-10-14 19:23:37,277 - stpipe.Spec2Pipeline.residual_fringe - INFO - Number of columns corrected for channel 0
2022-10-14 19:23:37,278 - stpipe.Spec2Pipeline.residual_fringe - INFO - Processing complete
2022-10-14 19:23:37,279 - stpipe.Spec2Pipeline.residual_fringe - INFO -  adding units back to output array
2022-10-14 19:23:37,299 - stpipe.Spec2Pipeline.residual_fringe - INFO - Step residual_fringe done
2022-10-14 19:23:37,698 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage2/jw02732005001_02105_00002_mirifushort_cal.fits>,).
2022-10-14 19:23:37,700 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build para

In [20]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 22159.9792 seconds


5.<font color='white'>-</font>Spec3 Pipeline: Default configuration (4 per-channel cubes)<a class="anchor" id="spec3"></a>
------------------

<div class="alert alert-block alert-warning">
Here we'll run the Spec3 pipeline to produce a composite data cube from all dithered exposures.
We will need to create an association file from all science and background data in order for the pipeline to use them appropriately.

A word of caution: the data cubes created by the JWST pipeline are in SURFACE BRIGHTNESS units (MJy/steradian), not flux units.  What that means is that if you intend to sum spectra within an aperture you need to be sure to multiply by the pixel area in steradians first in order to get a spectrum in flux units (the PIXAR_SR keyword can be found in the SCI extension header).  This correction is already build into the pipeline Extract1D algorithm.

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html
    
</div>

In [21]:
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    nbg=len(bgfiles)
    for ii in range(0,nbg):
        asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [22]:
# Find and sort all of the input files

# Science Files need the cal.fits files
sstring = spec2_dir + 'jw*mirifu*cal.fits'
calfiles = np.array(sorted(glob.glob(sstring)))

# Background Files need the x1d.fits files
sstring = spec2_bgdir + 'jw*mirifu*x1d.fits'
bgfiles = np.array(sorted(glob.glob(sstring)))

print('Found ' + str(len(calfiles)) + ' science files to process')
print('Found ' + str(len(bgfiles)) + ' background files to process')

Found 24 science files to process
Found 12 background files to process


In [23]:
# Make an association file that includes all of the different exposures
asnfile=os.path.join(output_dir, 'l3asn.json')
if dospec3:
    writel3asn(calfiles, bgfiles, asnfile, 'Level3')

2022-10-14 19:25:18,736 - stpipe - WARNING - /Users/dlaw/jwcode/jwst/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)



In [24]:
# Define a function that will call the spec3 pipeline with our desired set of parameters
# This is designed to run on an association file
def runspec3(filename):
    # This initial setup is just to make sure that we get the latest parameter reference files
    # pulled in for our files.  This is a temporary workaround to get around an issue with
    # how this pipeline calling method works.
    crds_config = Spec3Pipeline.get_config_from_reference(filename)
    spec3 = Spec3Pipeline.from_config_section(crds_config)
    
    spec3.output_dir = spec3_dir
    spec3.save_results = True
    
    # Cube building configuration options
    # spec3.cube_build.output_file = 'bandcube' # Custom output name
    # spec3.cube_build.output_type = 'band' # 'band', 'channel', or 'multi' type cube output
    # spec3.cube_build.channel = '1' # Build everything from just channel 1 into a single cube (we could also choose '2','3','4', or 'ALL')
    # spec3.cube_build.weighting = 'drizzle' # 'emsm' or 'drizzle'
    # spec3.cube_build.coord_system = 'ifualign' # 'ifualign', 'skyalign', or 'internal_cal'
    # spec3.cube_build.scale1 = 0.5 # Output cube spaxel scale (arcsec) in dimension 1 if setting it by hand
    # spec3.cube_build.scale2 = 0.5 # Output cube spaxel scale (arcsec) in dimension 2 if setting it by hand
    # spec3.cube_build.scalew = 0.002 # Output cube spaxel size (microns) in dimension 3 if setting it by hand
    
    # Overrides for whether or not certain steps should be skipped
    #spec3.assign_mtwcs.skip = False
    #spec3.master_background.skip = True
    #spec3.outlier_detection.skip = True
    #spec3.mrs_imatch.skip = True
    #spec3.cube_build.skip = False
    #spec3.extract_1d.skip = False
    
    # Cubepar overrides
    #spec3.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec3.extract1d.override_extract1d = 'myfile.asdf'
    #spec3.extract1d.override_apcorr = 'myfile.asdf'

    spec3(filename)

In [25]:
if dospec3:
    runspec3(asnfile)
else:
    print('Skipping Spec3 processing')

2022-10-14 19:25:20,846 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: /grp/crds/jwst/references/jwst/jwst_miri_pars-outlierdetectionstep_0051.asdf
2022-10-14 19:25:20,876 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: /grp/crds/jwst/references/jwst/jwst_miri_pars-spec3pipeline_0002.asdf
2022-10-14 19:25:20,909 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2022-10-14 19:25:20,911 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2022-10-14 19:25:20,913 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2022-10-14 19:25:20,914 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSIMatchStep instance created.
2022-10-14 19:25:20,917 - stpipe.Spec3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2022-10-14 19:25:20,919 - stpipe.Spec3Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2022-10-14 19:25:20,922 - stpipe.Spec3Pipeline.cube_build - INFO - 

2022-10-14 19:26:16,460 - stpipe.Spec3Pipeline.master_background - INFO - Using exposure time as the weight.
2022-10-14 19:26:18,809 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 1
2022-10-14 19:26:18,832 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 2
2022-10-14 19:26:18,866 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 3
2022-10-14 19:26:18,889 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 4
2022-10-14 19:26:18,923 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 5
2022-10-14 19:26:18,942 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 6
2022-10-14 19:26:18,975 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 7
2022-10-14 19:26:18,994 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data f

2022-10-14 19:29:02,561 - stpipe.Spec3Pipeline.outlier_detection - INFO - Working with instrument MIRI
2022-10-14 19:29:02,562 - stpipe.Spec3Pipeline.outlier_detection - INFO - Shape of sky cube 33.000000 33.000000 632.000000
2022-10-14 19:29:02,562 - stpipe.Spec3Pipeline.outlier_detection - INFO - Channel 4
2022-10-14 19:29:02,563 - stpipe.Spec3Pipeline.outlier_detection - INFO - Sub-channel medium
2022-10-14 19:29:04,387 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02103_00001_mirifulong_cal.fits
2022-10-14 19:29:06,726 - stpipe.Spec3Pipeline.outlier_detection - WARNING - /Users/dlaw/anaconda3/envs/jwst/lib/python3.10/site-packages/gwcs/selector.py:592: RuntimeWarning: invalid value encountered in add
  result = [np.empty(inputs[0].shape) +

2022-10-14 19:29:06,890 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02103_00002_mirifulong_cal.fits
2022-10-14 19:29:09,348 - stpipe.Spec3Pipeline.outlier_detection - INFO -

2022-10-14 19:29:59,048 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02103_00004_mirifushort_cal.fits
2022-10-14 19:30:02,291 - stpipe.CubeBuildStep - INFO - CubeBuildStep instance created.
2022-10-14 19:30:02,292 - stpipe.CubeBuildStep - INFO - Starting IFU Cube Building Step
2022-10-14 19:30:02,292 - stpipe.CubeBuildStep - INFO - Input interpolation: pointcloud
2022-10-14 19:30:02,293 - stpipe.CubeBuildStep - INFO - Coordinate system to use: skyalign
2022-10-14 19:30:02,293 - stpipe.CubeBuildStep - INFO - Weighting method for point cloud: emsm
2022-10-14 19:30:02,294 - stpipe.CubeBuildStep - INFO - Power weighting distance: 2.0
2022-10-14 19:30:02,295 - stpipe.CubeBuildStep - INFO - Cube Type: Single cubes
2022-10-14 19:30:02,649 - stpipe.Spec3Pipeline.outlier_detection - INFO - The desired cubes cover the MIRI Channels: ['3']
2022-10-14 19:30:02,649 - stpipe.Spec3Pipeline.outlier_detection - INFO - The desired cubes cover the MIRI subchannels: ['l

2022-10-14 19:30:49,601 - stpipe.Spec3Pipeline.outlier_detection - INFO - Single = true, creating a set of single exposures mapped to output IFUCube coordinate system
2022-10-14 19:30:49,611 - stpipe.Spec3Pipeline.outlier_detection - INFO - Cube Geometry:
2022-10-14 19:30:49,612 - stpipe.Spec3Pipeline.outlier_detection - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(arcsec)  Min & Max (xi, eta arcsec)
2022-10-14 19:30:49,612 - stpipe.Spec3Pipeline.outlier_detection - INFO - Axis 1    43  22.00 -20.98483715   0.17000000  -3.65500004   3.65500004
2022-10-14 19:30:49,613 - stpipe.Spec3Pipeline.outlier_detection - INFO - Axis 2    39  20.00  33.97593269   0.17000000  -3.31500003   3.31500003
2022-10-14 19:30:49,614 - stpipe.Spec3Pipeline.outlier_detection - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(microns)  Min & Max (microns)
2022-10-14 19:30:49,614 - stpipe.Spec3Pipeline.outlier_detection - INFO - Axis 3   840   1.00  10.02100046   0.00200000  10.02000046  11.70000054
2022-10-14 1

...

2022-10-14 19:35:49,721 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02101_00002_mirifulong_cal.fits
2022-10-14 19:35:56,916 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02101_00003_mirifulong_cal.fits
2022-10-14 19:36:00,402 - stpipe.Spec3Pipeline.outlier_detection - WARNING - /Users/dlaw/anaconda3/envs/jwst/lib/python3.10/site-packages/gwcs/selector.py:592: RuntimeWarning: invalid value encountered in add
  result = [np.empty(inputs[0].shape) +

2022-10-14 19:36:00,747 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02101_00004_mirifulong_cal.fits
2022-10-14 19:36:03,182 - stpipe.CubeBuildStep - INFO - CubeBuildStep instance created.
2022-10-14 19:36:03,183 - stpipe.CubeBuildStep - INFO - Starting IFU Cube Building Step
2022-10-14 19:36:03,184 - stpipe.CubeBuildStep - INFO - Input interpolation: pointcloud
2022-10-14 19:36:03,185 - stpipe.CubeBuildStep - INFO - Coordinate system t

2022-10-14 19:36:47,282 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 2937 (0.28%)
2022-10-14 19:36:47,388 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 1714 (0.16%)
2022-10-14 19:36:47,496 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 2664 (0.25%)
2022-10-14 19:36:47,601 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 1489 (0.14%)
2022-10-14 19:36:47,708 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 2393 (0.23%)
2022-10-14 19:36:47,814 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 1011 (0.10%)
2022-10-14 19:36:47,919 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 4095 (0.39%)
2022-10-14 19:36:48,025 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 927 (0.09%)
2022-10-14 19:36:48,133 - stpipe.Spec3Pipeline.ou

2022-10-14 19:37:40,922 - stpipe.Spec3Pipeline.cube_build - INFO - Starting IFU Cube Building Step
2022-10-14 19:37:40,923 - stpipe.Spec3Pipeline.cube_build - INFO - Input interpolation: drizzle
2022-10-14 19:37:40,923 - stpipe.Spec3Pipeline.cube_build - INFO - Coordinate system to use: skyalign
2022-10-14 19:37:41,285 - stpipe.Spec3Pipeline.cube_build - INFO - The desired cubes cover the MIRI Channels: ['1', '1', '1', '2', '2', '2', '3', '3', '3', '4', '4', '4']
2022-10-14 19:37:41,286 - stpipe.Spec3Pipeline.cube_build - INFO - The desired cubes cover the MIRI subchannels: ['short', 'medium', 'long', 'short', 'medium', 'long', 'short', 'medium', 'long', 'short', 'medium', 'long']
2022-10-14 19:37:41,287 - stpipe.Spec3Pipeline.cube_build - INFO - Reading cube parameter file /grp/crds/jwst/references/jwst/jwst_miri_cubepar_0009.fits
2022-10-14 19:37:42,011 - stpipe.Spec3Pipeline.cube_build - INFO - Output cubes are single channel and all subchannels in data
2022-10-14 19:37:42,012 - stp

2022-10-14 19:41:01,250 - stpipe.Spec3Pipeline.cube_build - INFO - Total # of holes for IFU cube is : 0
2022-10-14 19:41:01,277 - stpipe.Spec3Pipeline.cube_build - INFO - Number of spectral tear planes adjusted: 0
2022-10-14 19:41:04,645 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-10-14 19:41:04,937 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-10-14 19:41:05,230 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-10-14 19:41:05,520 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-10-14 19:41:05,813 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-10-14 19:41:06,108 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-10-14 19:41:06,398 - stpipe.Spec3Pipe

2022-10-14 19:44:30,045 - stpipe.Spec3Pipeline.cube_build - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/Level3_ch1-shortmediumlong_s3d.fits
2022-10-14 19:44:31,053 - stpipe.Spec3Pipeline.cube_build - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/Level3_ch4-shortmediumlong_s3d.fits
2022-10-14 19:44:31,054 - stpipe.Spec3Pipeline.cube_build - INFO - Step cube_build done
2022-10-14 19:44:32,642 - stpipe.Spec3Pipeline.extract_1d - INFO - Step extract_1d running with args (<ModelContainer>,).
2022-10-14 19:44:32,645 - stpipe.Spec3Pipeline.extract_1d - INFO - Step extract_1d parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'x1d', 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04', 'smoothing_length': None, 'bkg_fit': No

In [26]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 23566.8903 seconds


<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://www.stsci.edu/~dlaw/stsci_logo.png" alt="stsci_logo" width="200px"/> 